# 모델 불러오고 추론하기

In [ ]:
# uv add huggingface_hub
# 환경변수 불러오기
from dotenv import load_dotenv
load_dotenv()

from huggingface_hub import login
import os
login(token=os.getenv("HUGGINGFACE_API_KEY"))

In [ ]:
import torch
print(torch.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

2.8.0+cu126
cuda


# 1) Gemma3 1B it 모델 불러오기

### pipeline으로 불러오기

In [ ]:
# uv add transformers
from transformers import pipeline
import torch

# 파이프라인 만들기
pipe = pipeline(
    "text-generation",
    model="google/gemma-3-1b-it",
    device=device,
    torch_dtype=torch.bfloat16
)

# 입력값 만들기
messages = [
    {
        "role": "system",
        "content": "친절하게 말하세요"
    },
    {
        "role": "user",
        "content": "오늘 하루를 응원해주세요"
    }
]
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "친절하게 말하세요"}]
    },
    {
        "role": "user",
        "content": [{"type": "text", "text": "오늘 하루를 응원해주세요"}]
    }
]

output = pipe(messages, max_new_tokens=50)
print(output)

Device set to use cuda


[{'generated_text': [{'role': 'system', 'content': [{'type': 'text', 'text': '친절하게 말하세요'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '오늘 하루를 응원해주세요'}]}, {'role': 'assistant', 'content': '안녕하세요! 오늘 하루도 따뜻하고 행복하게 보내시길 바랍니다. \n\n힘든 하루였을 텐데, 잠시 숨을 고르고, 오늘 하루를 긍정적으로 보내는 데 집중해 보세요. \n\n*'}]}]


In [ ]:
# 정리 1. output은 어떻게 생겼을까
# 정리 2. messages 형태는 왜 이렇게 이루어져 있을까?

In [ ]:
# 미션 output 파싱해보기
for out in output:
    print(out)
    print(out["generated_text"])
    for x in out["generated_text"]:
        print(x)

{'generated_text': [{'role': 'system', 'content': [{'type': 'text', 'text': '친절하게 말하세요'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '오늘 하루를 응원해주세요'}]}, {'role': 'assistant', 'content': '안녕하세요! 오늘 하루도 따뜻하고 행복하게 보내시길 바랍니다. \n\n힘든 하루였을 텐데, 잠시 숨을 고르고, 오늘 하루를 긍정적으로 보내는 데 집중해 보세요. \n\n*'}]}
[{'role': 'system', 'content': [{'type': 'text', 'text': '친절하게 말하세요'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '오늘 하루를 응원해주세요'}]}, {'role': 'assistant', 'content': '안녕하세요! 오늘 하루도 따뜻하고 행복하게 보내시길 바랍니다. \n\n힘든 하루였을 텐데, 잠시 숨을 고르고, 오늘 하루를 긍정적으로 보내는 데 집중해 보세요. \n\n*'}]
{'role': 'system', 'content': [{'type': 'text', 'text': '친절하게 말하세요'}]}
{'role': 'user', 'content': [{'type': 'text', 'text': '오늘 하루를 응원해주세요'}]}
{'role': 'assistant', 'content': '안녕하세요! 오늘 하루도 따뜻하고 행복하게 보내시길 바랍니다. \n\n힘든 하루였을 텐데, 잠시 숨을 고르고, 오늘 하루를 긍정적으로 보내는 데 집중해 보세요. \n\n*'}


In [ ]:
# 입력값 만들기
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "친절하게 말하세요"}]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "오늘 하루를 응원해주세요"}]
        }
    ]
]

output = pipe(messages, max_new_tokens=50)
print(output)

[[{'generated_text': [{'role': 'system', 'content': [{'type': 'text', 'text': '친절하게 말하세요'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '오늘 하루를 응원해주세요'}]}, {'role': 'assistant', 'content': '안녕하세요! 😊 오늘 하루를 응원해 드릴게요. \n\n힘든 하루였을 텐데, 잠시 숨을 고르고, 오늘 하루를 긍정적으로 보내시길 바랍니다. \n\n*   **작은 것'}]}]]


### 모델로 불러오기

In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch
# 모델 이름 설정
model_id = "google/gemma-3-1b-it"

# STEP1. 모델, 토크나이저 불러오기
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# STEP2. 입력 데이터 준비하기
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Write a poem on Hugging Face, the company"},]
        },
    ],
]
# STEP3. 입력 데이터 토크나이징하기
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
)

# STEP4. 추론하기
with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=64)

outputs = tokenizer.batch_decode(outputs)


In [ ]:
test_dict = {"device": "cuda", "verbose": True}

temp_func(**test_dict)

temp_func(device="cuda", verbose=True)

In [ ]:
# uv add accelerate bitsandbytes
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch
# 모델 이름 설정
model_id = "google/gemma-3-1b-it"

# STEP1. 모델, 토크나이저 불러오기
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# STEP2. 입력 데이터 준비하기
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Write a poem on Hugging Face, the company"},]
        },
    ],
]
# STEP3. 입력 데이터 토크나이징하기
# 테스트해보기 1. (add_generation_prompt=False, tokenize=False)
# 테스트해보기 2. (add_generation_prompt=True, tokenize=False)
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # input뒤에다가 assistant를 붙일지 결정
    tokenize=True,              # 결과를 토큰화할지 여부
    return_dict=True,           # 결과를 딕셔너리로 반환하게 할 것인지의 여부
    return_tensors="pt",        # 결과를 파이토치 형식으로 반환할 것인지의 여부
)
print(inputs)

{'input_ids': tensor([[     2,    105,   2364,    107,   3048,    659,    496,  11045,  16326,
         236761,    108,   6974,    496,  27355,    580,  22798,   3801,   7117,
         236764,    506,   2544,    106,    107,    105,   4368,    107]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}


In [ ]:
# 정리1. messages변수가 토크나이저를 만나 숫자로 바뀌는 과정은 어떤가?
messages = [
    {},
    {}
]
# 정리2. 우리가 예측을 하기 위해서는 어떤 데이터가 준비되어야 할까?
# 정리3. 예측을 하는 과정에서 "**"는 왜 쓰는걸까?
# 정리4. output은 어떻게 나오는가?
# 정리5. ouutput에서 답변은 어떻게 추출할 수 있을까?

## messages를 토크나이저를 통해 숫자로 바꿔야 한다!
## tokenizer.apply_chat_template이라는 함수를 이용, tokenize=False ---- messages가 문자로 바뀐 상태가 나온다.
## tokenize=True로 설정하게 되면, input_ids, attention_mask 딕셔너리로 출력된다.
## input_ids : 문자 -> 숫자 , attention_mask: 그 자리가 의미가 있는 자리인지 의미가 없는 자리인지를 알려줌
## 이제 inputs가 준비되었다 -- GPU에 옮긴다 to(device) -- model.generate()
## outputs가 나온다. 이 친구는 어떻게 생겼을까? 답변이 어디있는지 찾을 수 있는가??

In [ ]:
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False, # input뒤에다가 assistant를 붙일지 결정
    tokenize=False,              # 결과를 토큰화할지 여부
)
print(inputs[0])

<bos><start_of_turn>user
You are a helpful assistant.

Write a poem on Hugging Face, the company<end_of_turn>



In [ ]:
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # input뒤에다가 assistant를 붙일지 결정
    tokenize=False,              # 결과를 토큰화할지 여부
)
print(inputs[0])

<bos><start_of_turn>user
You are a helpful assistant.

Write a poem on Hugging Face, the company<end_of_turn>
<start_of_turn>model



In [ ]:
inputs

{'input_ids': tensor([[     2,    105,   2364,    107,   3048,    659,    496,  11045,  16326,
         236761,    108,   6974,    496,  27355,    580,  22798,   3801,   7117,
         236764,    506,   2544,    106,    107,    105,   4368,    107]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [ ]:
model.generate(input_ids, attention_mask, max_new_tokens)

In [ ]:
# STEP4. 추론하기
inputs = inputs.to(device)

with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=64)

outputs = tokenizer.batch_decode(outputs)
print(outputs)

['<bos><start_of_turn>user\nYou are a helpful assistant.\n\nWrite a poem on Hugging Face, the company<end_of_turn>\n<start_of_turn>model\nOkay, here’s a poem about Hugging Face, aiming to capture its essence – a community of AI, collaboration, and open-source creativity:\n\n**The Hugging Face Bloom**\n\nThe cloud expands, a gentle hue,\nHugging Face, a vibrant view.\nA haven built for code and']


In [ ]:
print(outputs[0])

<bos><start_of_turn>user
You are a helpful assistant.

Write a poem on Hugging Face, the company<end_of_turn>
<start_of_turn>model
Okay, here’s a poem about Hugging Face, aiming to capture its essence – a community of AI, collaboration, and open-source creativity:

**The Hugging Face Bloom**

The cloud expands, a gentle hue,
Hugging Face, a vibrant view.
A haven built for code and


In [ ]:
inputs

{'input_ids': tensor([[     2,    105,   2364,    107,   3048,    659,    496,  11045,  16326,
         236761,    108,   6974,    496,  27355,    580,  22798,   3801,   7117,
         236764,    506,   2544,    106,    107,    105,   4368,    107]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]], device='cuda:0')}

In [ ]:
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # input뒤에다가 assistant를 붙일지 결정
    tokenize=True,              # 결과를 토큰화할지 여부
    return_dict=True,           # 결과를 딕셔너리로 반환하게 할 것인지의 여부
    return_tensors="pt",        # 결과를 파이토치 형식으로 반환할 것인지의 여부
)
input_len = inputs["input_ids"].shape[1]
print(outputs[0][input_len:])

ou are a helpful assistant.

Write a poem on Hugging Face, the company<end_of_turn>
<start_of_turn>model
Okay, here’s a poem about Hugging Face, aiming to capture its essence – a community of AI, collaboration, and open-source creativity:

**The Hugging Face Bloom**

The cloud expands, a gentle hue,
Hugging Face, a vibrant view.
A haven built for code and


## 2) Gemma3 1B pt 모델 불러오기

### pipeline으로 불러오기

In [29]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation", 
    model="google/gemma-3-1b-pt", 
    device=device, 
    torch_dtype=torch.bfloat16
)

output = pipe("Eiffel tower is located in", max_new_tokens=50)


Device set to use cuda


In [28]:
print(output)
print(output[0])
print(output[0]["generated_text"])

[{'generated_text': 'Eiffel tower is located in the 7th district of ParisThe Eiffel Tower is located in the 7th district of Paris, FranceThe Eiffel Tower is the world’s tallest towerThe Eiffel Tower is the tallest tower in ParisEiffel Tower is located'}]
{'generated_text': 'Eiffel tower is located in the 7th district of ParisThe Eiffel Tower is located in the 7th district of Paris, FranceThe Eiffel Tower is the world’s tallest towerThe Eiffel Tower is the tallest tower in ParisEiffel Tower is located'}
Eiffel tower is located in the 7th district of ParisThe Eiffel Tower is located in the 7th district of Paris, FranceThe Eiffel Tower is the world’s tallest towerThe Eiffel Tower is the tallest tower in ParisEiffel Tower is located


### 모델로 불러오기

In [ ]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM
# 모델 이름 설정
model_id = "google/gemma-3-1b-pt"

# STEP1. 모델, 코트나이저 불러오기
model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# STEP2. 입력 데이터 준비하기
prompt = "Eiffel tower is located in"

# STEP3. 입력 데이터 토크나이징하기
inputs = tokenizer(
    prompt, 
    return_tensors="pt"
)
inputs = inputs.to(device) # GPU 보내기

# STEP4. 추론하기
input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=50, 
        do_sample=False
    )

outputs = tokenizer.decode(outputs)


In [30]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM
# 모델 이름 설정
model_id = "google/gemma-3-1b-pt"

# STEP1. 모델, 코트나이저 불러오기
model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

`torch_dtype` is deprecated! Use `dtype` instead!


In [34]:
# STEP2. 입력 데이터 준비하기
prompt = "Eiffel tower is located in"

# STEP3. 입력 데이터 토크나이징하기
inputs = tokenizer(
    prompt, 
    return_tensors="pt"
)
print(inputs)
inputs = inputs.to(device) # GPU 보내기

{'input_ids': tensor([[     2, 236788,  80880,  18515,    563,   5628,    528]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [41]:
# STEP4. 추론하기
input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=50, 
        do_sample=False
    )
print(outputs)
print(outputs.shape)
print(outputs[0])

output = tokenizer.decode(outputs[0])
print(output)

tensor([[     2, 236788,  80880,  18515,    563,   5628,    528,    506,   3710,
            529,   9079, 236764,   7001, 236761, 255999,    818,  94648,  25822,
            563,    496, 236743, 236800, 236778, 236812, 236772,  33307, 236772,
          11480,  18515,    528,   9079, 236764,   7001, 236761, 255999,    818,
          94648,  25822,    563,    496,   5404,    529,   9079,    532,   7001,
         236761, 255999,    818,  94648,  25822,    563,    496,   5404,    529,
           9079,    532,   7001]], device='cuda:0')
torch.Size([1, 57])
tensor([     2, 236788,  80880,  18515,    563,   5628,    528,    506,   3710,
           529,   9079, 236764,   7001, 236761, 255999,    818,  94648,  25822,
           563,    496, 236743, 236800, 236778, 236812, 236772,  33307, 236772,
         11480,  18515,    528,   9079, 236764,   7001, 236761, 255999,    818,
         94648,  25822,    563,    496,   5404,    529,   9079,    532,   7001,
        236761, 255999,    818,  94648,  2

In [42]:
print(tokenizer.batch_decode(outputs))

['<bos>Eiffel tower is located in the heart of Paris, France.<start_of_image>The Eiffel Tower is a 324-meter-high tower in Paris, France.<start_of_image>The Eiffel Tower is a symbol of Paris and France.<start_of_image>The Eiffel Tower is a symbol of Paris and France']


In [ ]:
# 정리
# 1. 우리가 허깅페이스에서 LLM 모델을 사용하려고 할 때, 모델은 어떻게 불러오나요?
# 2. instruct 모델이 있고, 그냥 pre-trained 모델이 있는데 모델에 input해야 할 데이터는 어떻게 생겼나요?
# 3. input 해야할 데이터는 어떻게 만드나요?
# 4. model에서 input data를 넣은 다음 나온 output은 어떻게 생겼나요?
# 5. output을 텍스트로 바꾸려면 어떻게 해야 하나요?

In [47]:
print(tokenizer.eos_token_id)
print(tokenizer.bos_token_id)

1
2


In [ ]:
# 응용해보기(업데이트 중입니다.......)
prompt = [
    "남산타워는 어디에 있나요?<eos>",
    "경복궁은 어디에 있나요?<eos>"
]
prompt = "남산타워는 어디에 있나요?\n경복궁은 어디에 있나요?"

# STEP3. 입력 데이터 토크나이징하기
inputs = tokenizer(
    "".join(prompt), 
    return_tensors="pt"
)
print(inputs)
inputs = inputs.to(device) # GPU 보내기

{'input_ids': tensor([[     2, 239120, 238500, 238503, 239592, 237170, 110388, 237223,   5386,
         103595, 236881,    107, 238136, 239484, 241845, 237456, 110388, 237223,
           5386, 103595, 236881]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
## Gemme-3 1b it

# 정리1. messages변수가 토크나이저를 만나 숫자로 바뀌는 과정은 어떤가?
messages = [
    [
        {
            "role": "system",
            "content": [
                {"type": "text", "text": "You are a helpful assistant."},
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Write a poem on Hugging Face, the company"},
            ],
        },
    ],
]
## 모델에 바로 넣을 수 있는가????? No
## messages 를 숫자 텐서로 바꿔야한다. ---- How?? tokenize.apply_chat_template(message)
## 만약 tokenize=True를 하면, {"input_ids":, "attention_mask":}
## ??? messages 는 문자가 아닌데 어떻게 숫자로 바꾸는거지?????? tokenize = False 해보면 알 수 있다.
##  <bos><start....... bos 는 시작, eos 끝

# 정리2. 우리가 예측을 하기 위해서는 어떤 데이터가 준비되어야 할까?
## inputs = {"input_ids":"A", "attention_mask":"B"}

# 정리3. 예측을 하는 과정에서 "**"는 왜 쓰는걸까?
## def myfunc(input_ids, attention_mask, message="CCC", verbose = Fase) 
## ...
## myfunc(**inputs)의 의미는 myfunc(input_ids = "A", attention_mask="B") 로 해주세요와 같다.

# 정리4. output은 어떻게 나오는가?
## outputs = model(inputs)
## tensor - 숫자형

# 정리5. ouutput에서 답변은 어떻게 추출할 수 있을까? 
# decode를 통해서 숫자 텐서를 텍스트로 바꿩한다.
# tensor([데이터1, 데이터2 ...]) 인 경우, tokenizer.batch_decode
# tensor(데이터1)인 경우, tokenizer.decode

In [ ]:
## Gemma-3 1b pt

# 정리
# 1. 우리가 허깅페이스에서 LLM 모델을 사용하려고 할 때, 모델은 어떻게 불러오나요?
## 모델 이름이 있는 사이트에 들어가면 
## pipeline 함수로도 추론할 수 있고,
## model을 직접 불러와서 할 수도 있다.----(별표) 파인튜닝: 내 데이터를 이미 학습되어 있는 모델에 적용시키기 위해 model 

# 2. instruct 모델이 있고, 그냥 pre-trained 모델이 있는데 모델에 input해야 할 데이터는 어떻게 생겼나요?
## LLM 학습 방법 
## 1) Pre-traind model 에서 "새로운 정보"를 학습시킨다. ex. gemma-3-1b-pt INPUT: "프롬프트를 작성해주세요" (str)
## 2) Instruct model 에서 "말하는 방식"을 학습시킨다. ex. gemma-3-1b-it INPUT: 대화 [{"role":,"content":}] * 지시사항까지 학습한다.

# 3. input 해야할 데이터는 어떻게 만드나요?
## 텍스트 -- 텐서 -- model -- 텍스트
##    tokenizer       tokenizer
##    tokenize()      tokenizer.decode()
##    tokenize.apply_chat_template()  

# 4. model에서 input data를 넣은 다음 나온 output은 어떻게 생겼나요?
## tenser 

# 5. output을 텍스트로 바꾸려면 어떻게 해야 하나요?
# tokenizer.decode 시리즈 활용